In [1]:
from datasets import list_datasets, load_dataset, load_dataset_builder
from transformers import AutoTokenizer

In [2]:
ds_builder = load_dataset_builder("sst2")

/Users/vale/opt/anaconda3/envs/torch_env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
ds_builder.info.description
ds_builder.info.features

{'idx': Value(dtype='int32', id=None),
 'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [4]:
train_data = load_dataset("sst2", split="train")
test_data = load_dataset("sst2", split="test")

Found cached dataset sst2 (/Users/vale/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
Found cached dataset sst2 (/Users/vale/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased",)

In [6]:
tokenizer(train_data[0]["sentence"], padding=True, max_length=10, truncation=True)

{'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
def tokenization(example):
    return tokenizer(example["sentence"],  padding=True, max_length=128, truncation=True)

dataset = train_data.map(tokenization, batched=True)

Loading cached processed dataset at /Users/vale/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5/cache-5f61e4dd47a32050.arrow


In [8]:
dataset.format

{'type': None,
 'format_kwargs': {},
 'columns': ['idx',
  'sentence',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 'output_all_columns': False}

In [9]:
dataset.set_format(type="torch", columns=["idx", "input_ids", "label"])
dataset.format['type']

'torch'

In [10]:
dataset[122]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'idx': tensor(122),
 'label': tensor(0),
 'input_ids': tensor([  101, 22902,  7982,  1010,  3082,  1011,  4375,  3042,  2100,  1011,
          3110, 15792,  1010,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0])}

In [11]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=128)

In [12]:
from src.model import make_model

In [13]:
test_model = make_model(10, 10, 1)

In [14]:
dataset[1]

{'idx': tensor(1),
 'label': tensor(0),
 'input_ids': tensor([  101,  3397,  2053, 15966,  1010,  2069,  4450,  2098, 18201,  2015,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0])}

In [15]:
test_model(dataset[1])

TypeError: EncoderDecoder.forward() missing 3 required positional arguments: 'tgt', 'src_mask', and 'tgt_mask'